In [1]:
#!pip install mysql
#!pip install mysql-connector-python-rf
#import warnings
#warnings.filterwarnings("ignore")

You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from flask import Flask
from flask import url_for, render_template, request, redirect
import mysql.connector as mysql

In [12]:
session = {}
session['username'] = None
global_data = []

In [13]:
app = Flask(__name__)

In [14]:
con = mysql.connect(user = 'root', password = 'password', 
                    host = '127.0.0.1', database = 'db_proj', #database = 'task_manager'
                   auth_plugin='mysql_native_password')
cur = con.cursor(dictionary=True)

In [15]:
@app.route('/home', methods=['GET', 'POST'])
def index():
    if session['username'] == None:
        return render_template('home.html')
    else:
        tasks = []
        importance = []
        statuses = []
        base_string = "SELECT task, importance, status FROM (SELECT tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance) as t"
        if request.method == "POST":
            det = request.form
##            with open ('test.txt', 'a') as f:
##                f.write(det['importance1'])
            importance_list = []
            status_list = []
            if 'importance1' in det:
                importance_list.append(det['importance1'])
            if 'importance2' in det:
                importance_list.append(det['importance2'])
            if 'importance3' in det:
                importance_list.append(det['importance3'])
                
            if 'status1' in det:
                status_list.append(det['status1'])
            if 'status2' in det:
                status_list.append(det['status2'])
            if 'status3' in det:
                status_list.append(det['status3'])
            with open ('test.txt', 'a') as f:
                f.write('after if')            
            if not importance_list and not status_list:
                cur.execute(base_string)
            else:
                base_string += ' WHERE ('
                if importance_list:
                    for i, element in enumerate(importance_list):
                        base_string = base_string + 'importance = \'' + element + '\''
                        if i != len(importance_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
                if status_list:
                    if importance_list:
                        base_string += ' AND ('
                    for i, element in enumerate(status_list):
                        base_string = base_string + 'status = \'' + element + '\''
                        if i != len(status_list) - 1:
                            base_string += ' OR '
                        else:
                            base_string += ')'
            cur.execute(base_string)  
            result = cur.fetchall()
        else:
            cur.execute(base_string)
            result = cur.fetchall()

        for res in result:
            tasks.append(res['task'])
            importance.append(res['importance'])
            statuses.append(res['status'])
            task_imp_stats = zip(tasks, importance, statuses)
        return render_template('home_logged_in.html', tasks=tasks, importance=importance, 
                               statuses=statuses, task_imp_stats=task_imp_stats)

In [16]:
@app.route('/update', methods=['GET', 'POST'])
def update_task():
    task_id = request.args.get('id')
    if request.method == "POST":
        details = request.form
        cur.execute("SELECT id_user FROM users_list WHERE username = '" + session['username'] + "'")
        res = cur.fetchone()
        id_user = res['id_user']
        
        cur.execute("SELECT id_status FROM status_list WHERE status_name = '" + details['status'] + "'")
        res = cur.fetchone()
        id_status = res['id_status']
        
        cur.execute("SELECT id_importance FROM importance_list WHERE importance_name = '" + details['importance'] + "'")
        res = cur.fetchone()
        id_importance = res['id_importance']
        
        #cur.execute('SET SQL_SAFE_UPDATES = 0') # Это чтоб давал апдейтить, возможно тут не пригодится, в воркбенче было надо
        cur.execute("UPDATE user_tasks SET id_status = " + id_status + ", id_importance = " + id_importance +
                    "WHERE id_user_task =" + task_id)
        #cur.execute('SET SQL_SAFE_UPDATES = 1') # Это чтоб вернуть запрет на всяческие апдейты непредвиденные))
        con.commit()
        return redirect('home')
    
    importance = []
    status = []
    cur.execute("SELECT importance_name FROM importance_list")
    res = cur.fetchall()
    for i in res:
        importance.append(i['importance_name'])
    cur.execute("SELECT status_name FROM status_list")
    res = cur.fetchall()
    for i in res:
        status.append(i['status_name'])
    return render_template('update_task.html', status = status, importance = importance, task=task)

In [17]:
@app.route('/signup', methods=['GET', 'POST'])
def sign_up():
    if request.method == "GET":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("INSERT INTO users_list (username, password) VALUES (%s, %s)", (username, password))
        con.commit()
        cur.close()
        return redirect('login')
    return render_template('sign_up.html')

In [18]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("SELECT * FROM users_list")
        users = cur.fetchall()
        user_exists = 0
        for i in users:
            if i['username'] == username:
                user_exists = 1
                if i['password'] == password:
                    session['username'] = username
                    return redirect('home')
                else:
                    return ('Неправильный пароль.')
        if user_exists == 1:
            return redirect ('home')
        else:
            return ('Такого пользователя пока нет.')
    return render_template('login.html')

In [19]:
@app.route('/new_task', methods=['GET', 'POST'])
def new_task(): 
    if request.method == "POST":
        details = request.form
        cur.execute("SELECT id_user FROM users_list WHERE username = '" + session['username'] + "'")
        res = cur.fetchone()
        id_user = res['id_user']

##        cur.execute("SELECT id_task FROM tasks_list WHERE task_name = '" + details['tasks'] + "'")
##        res = cur.fetchone()
##       id_task = res['id_task']
        
        cur.execute("SELECT id_status FROM status_list WHERE status_name = '" + details['status'] + "'")
        res = cur.fetchone()
        id_status = res['id_status']
        
        cur.execute("SELECT id_importance FROM importance_list WHERE importance_name = '" + details['importance'] + "'")
        res = cur.fetchone()
        id_importance = res['id_importance']
        
        cur.execute("SELECT task_name, id_task FROM tasks_list")
        res = cur.fetchall()
        task_exists = 0
        for i in res:
            if i['task_name'] == details['tasks']:
                task_exists = 1
                id_task = i['id_task']
        if task_exists == 0:        
            cur.execute("INSERT INTO tasks_list (task_name) VALUES ('" + details['tasks'] + "')")
            cur.execute("SELECT id_task FROM tasks_list WHERE task_name = '" + details['tasks'] + "'")
            res = cur.fetchone()
            id_task = res['id_task']
        data = (id_user, id_task, id_status, id_importance)
        cur.execute("INSERT INTO user_tasks (id_user, id_task, id_status, id_importance) VALUES (%s, %s, %s, %s)", data)
        con.commit()
        return redirect('home')
    importance = []
    status = []
    cur.execute("SELECT importance_name FROM importance_list")
    res = cur.fetchall()
    for i in res:
        importance.append(i['importance_name'])
    cur.execute("SELECT status_name FROM status_list")
    res = cur.fetchall()
    for i in res:
        status.append(i['status_name'])
    return render_template('new_task.html', status = status, importance = importance)

In [20]:
if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Dec/2019 01:54:09] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2019 01:54:19] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [25/Dec/2019 01:54:19] "GET /home HTTP/1.1" 200 -
[2019-12-25 01:54:24,658] ERROR in app: Exception on /update [GET]
Traceback (most recent call last):
  File "C:\Users\Sony\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Sony\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Sony\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Sony\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\Sony\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_requ

In [124]:
session['username'] = 'fishowl'
cur.execute("SELECT id_user FROM users_list WHERE username = '" + session['username'] + "'")

In [226]:
tasks = []
importance = []
statuses = []

cur.execute("SELECT tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance")
result = cur.fetchall()
result

[{'importance': 'очень важно',
  'status': 'в процессе',
  'task': 'вырезать снежинку'},
 {'importance': 'очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'},
 {'importance': 'не очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'},
 {'importance': 'важно',
  'status': 'не начато',
  'task': 'записаться к стоматологу'},
 {'importance': 'очень важно',
  'status': 'в процессе',
  'task': 'приготовить ужин'},
 {'importance': 'не очень важно',
  'status': 'в процессе',
  'task': 'приготовить ужин'},
 {'importance': 'не очень важно',
  'status': 'в процессе',
  'task': 'приготовить ужин'}]

In [231]:
cur.execute("SELECT tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + 'fishowl' + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance")
result = cur.fetchall()
result

[{'importance': 'очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'},
 {'importance': 'не очень важно',
  'status': 'не начато',
  'task': 'сделать домашку'}]

In [101]:
importance_list = ['важно', 'очень важно']
status_list = ['в процессе']
base_string = "SELECT task, importance, status FROM (SELECT tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance) as t"

if not importance_list and not status_list:
    cur.execute(base_string)
else:
    base_string += ' WHERE ('
    if importance_list:
        for i, element in enumerate(importance_list):
            base_string = base_string + 'importance = \'' + element + '\''
            if i != len(importance_list) - 1:
                base_string += ' OR '
            else:
                base_string += ')'
    if status_list:
        if importance_list:
            base_string += ' AND ('
        for i, element in enumerate(status_list):
            base_string = base_string + 'status = \'' + element + '\''
            if i != len(status_list) - 1:
                base_string += ' OR '
            else:
                base_string += ')'
        
cur.execute(base_string)  
result = cur.fetchall()

print(base_string)

SELECT task, importance, status FROM (SELECT tasks_list.task_name as 'task', importance_list.importance_name as 'importance',status_list.status_name as 'status' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list WHERE users_list.username = 'fishowl' and users_list.id_user = user_tasks.id_user and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status and importance_list.id_importance = user_tasks.id_importance) as t WHERE (importance = 'важно' OR importance = 'очень важно') AND (status = 'в процессе')


In [102]:
print(result)

[{'task': 'вырезать снежинку', 'importance': 'очень важно', 'status': 'в процессе'}]


In [83]:
cur.execute("SELECT task, importance, status FROM (SELECT tasks_list.task_name as 'task', importance_list.importance_name as 'importance',status_list.status_name as 'status' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list WHERE users_list.username = 'fishowl' and users_list.id_user = user_tasks.id_user and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status and importance_list.id_importance = user_tasks.id_importance) as t WHERE importance = 'важно'")
result = cur.fetchall()
result

[{'importance': 'важно',
  'status': 'не начато',
  'task': 'записаться к стоматологу'}]

In [81]:
cur.execute("SELECT task_name, id_task FROM tasks_list")
res = cur.fetchall()
res

[{'id_task': 1, 'task_name': 'помыть посуду'},
 {'id_task': 2, 'task_name': 'сделать домашку'},
 {'id_task': 3, 'task_name': 'приготовить ужин'},
 {'id_task': 4, 'task_name': 'записаться к стоматологу'},
 {'id_task': 5, 'task_name': 'вырезать снежинку'},
 {'id_task': 9, 'task_name': 'перебрать рис'}]

In [82]:
task_exists = 0
for i in res:
    if i['task_name'] == 'записаться к стоматологу':
        task_exists = 1
        id_task = i['id_task']
print(id_task)

4
